In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Install idx2numpy package for extracting data
!pip install idx2numpy ray

In [ ]:
# Import packages
import os
import gzip
import torch
import torchvision
import numpy as np 
import pandas as pd

import idx2numpy
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
# Import packages
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from functools import partial

In [ ]:
def load_one_dataset(path):
    '''
    Convenience function to load a single dataset
    '''
    f = gzip.open(path, 'rb')
    data = torch.from_numpy(idx2numpy.convert_from_file(f))
    f.close()
    
    return(data)


def load_all_datasets(train_imgs, train_labs, test_imgs, test_labs, batch_size):
    '''
    Load training as well as test images here
    '''
    train_images = load_one_dataset(train_imgs).type(torch.float32)
    train_labels = load_one_dataset(train_labs).type(torch.long)
    train = list(zip(train_images, train_labels))

    test_images = load_one_dataset(test_imgs).type(torch.float32)
    test_labels = load_one_dataset(test_labs).type(torch.long)
    test = list(zip(test_images, test_labels))
    
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=2)
    test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False, num_workers=2)
    
    return(train_loader, test_loader)

In [ ]:
class Net(nn.Module):
  
  def __init__(self, nb_units, input_dim, output_dim):
    '''
    Declare the network architecture here
    '''
    super(Net, self).__init__()
    
    # Initialize a list to store layers
    fc = []

    # Add input and output dimensions to layer list
    self.nb_units = [input_dim] + nb_units + [output_dim]

    # Now compute the total no. of layers
    self.nb_layers = len(self.nb_units)

    # Now append the hidden layers
    for i in range(1, self.nb_layers):
      fc.append(nn.Linear(self.nb_units[i-1], self.nb_units[i]))
    
    # Wrap this in a module list 
    self.fc = nn.ModuleList(fc)
    
  
  def forward(self, x):
    '''
    Send input forward through 
    the network
    '''
    # Reshape 28X28 images to be 784 X 784
    x = x.view(-1, 28*28)

    # Send example through network
    for layer in self.fc: x = F.relu(layer(x))
    
    return x

In [ ]:
def train(config, input_dim=784, output_dim = 10, epochs=2,
          data_dir = '/content/drive/MyDrive/data'):
    '''
    This is the main training loop
    '''
    # Set device
    if torch.cuda.is_available():
      device = torch.device("cuda")
    else:
      device = torch.device("cpu")
    
    # Set paths to datasets
    paths = {
        'train_imgs': os.path.join(data_dir, 'train-images-idx3-ubyte.gz'),
        'train_labs': os.path.join(data_dir, 'train-labels-idx1-ubyte.gz'),
        'test_imgs': os.path.join(data_dir,'t10k-images-idx3-ubyte.gz'),
        'test_labs': os.path.join(data_dir,'t10k-labels-idx1-ubyte.gz')
    }

    # Load datasets
    train_loader, test_loader = load_all_datasets(**paths, batch_size = config['batch_size'])
    
    # Set parameters
    net = Net(config['nb_units'], input_dim, output_dim)
    
    # Send net object to device memory
    net.to(device)
    
    # We use the cross-entropy loss
    criterion = nn.CrossEntropyLoss()

    # We use mini-batch stochastic gradient descent with momentum
    optimizer = optim.SGD(net.parameters(), lr=config['lr'], momentum=config['momentum'])

    # Loop over the dataset multiple times
    for epoch in range(epochs):  
        
        # Initialize running loss
        running_loss = 0.0
        running_accuracy = 0.0

        # Initialize the validation running loss
        val_running_loss = 0.0
        val_running_accuracy = 0.0
        
        # Iterate through data now
        for i, data in enumerate(train_loader):
            
            # Get the inputs: data is a list of [inputs, labels]
            inputs, labels = data
            
            # Send the inputs to the memory of the device
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward
            outputs = net(inputs)
            
            # Calculate loss
            loss = criterion(outputs, labels)

            # Get predictions
            _, preds = torch.max(outputs.data, 1)
            
            # Backward
            loss.backward()
            
            # Optimize
            optimizer.step()

            # Add to running loss
            running_loss += loss.item()

            # Add to running accuracy
            running_accuracy += (preds == labels).float().sum()
        
        # Loop through the validation data
        for j, data in enumerate(test_loader):
          
          # No need to calculate gradients for validation set
          with torch.no_grad():

              # Get the data item 
              val_inputs, val_labels = data
              val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)

              # Send the data item through the network to get output
              val_outputs = net(val_inputs)

              # Compute the loss
              val_loss = criterion(val_outputs, val_labels)

              # Get predictions
              _, val_preds = torch.max(val_outputs.data, 1)

              # Add to running loss
              val_running_loss += val_loss.item()

              # Add to running accuracy
              val_running_accuracy += (val_preds == val_labels).float().sum()
        
        # Rescale the training and validation perfomance metrics
        running_loss = running_loss/len(train_loader)
        running_accuracy = running_accuracy/(len(train_loader)*config['batch_size'])
        
        # Rescale the validation loss
        val_running_loss = val_running_loss/len(test_loader)
        val_running_accuracy = val_running_accuracy/(len(test_loader)*config['batch_size'])
        
        # Make print message format string
        msg = "{}, Epoch:{}, Loss:{}, Accuracy:{}," "\n"

        # Print performance
        print(msg.format("Training", epoch, running_loss, running_accuracy))
        print(msg.format("Validation", epoch, val_running_loss, val_running_accuracy))
        
        tune.report(loss= val_running_loss, accuracy= val_running_accuracy)

    # Print message
    print('Done training...')

In [ ]:
def get_random_hyper_parameters(n_layers=4):
    '''
    Randomly select hyper-parameters to train with
    '''
    config = {
        
      'lr': np.random.uniform(1e-4, 1e-1),
      'batch_size': 512,
      'weight_decay': np.random.uniform(1e-3, 1e-1),
      'nb_units': [2**np.random.randint(2,9) for i in range(n_layers)],
      'momentum': np.random.choice([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

    }
    
    return(config)

In [ ]:
def run_grid_search(num_samples=500, n_layers=4):

  for i in range(num_samples): 
    config = get_random_hyper_parameters(n_layers)
    results = train(config, epochs = 100)
  
  return(results)

{'batch_size': 512,
 'lr': 0.01,
 'momentum': 0.8,
 'nb_units': [256, 128, 64, 16],
 'weight_decay': 0.033018314247163325}

In [146]:
'''
config = {
        
      'lr': tune.grid_search([1e-4, 1e-3, 1e-2, 1e-1]),
      'batch_size': tune.grid_search([512, 1024]),
      'weight_decay': tune.grid_search([1e-3, 1e-2, 1e-1]),
      'nb_units': [132, 64, 32, 16],
      'momentum': tune.grid_search([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

}


config = {
        
      'lr': tune.grid_search([1e-4]),
      'batch_size': tune.grid_search([1024]),
      'weight_decay': tune.grid_search([1e-1]),
      'nb_units': tune.grid_search([[132, 64, 32, 16]]),
      'momentum': tune.grid_search([0.2])

}

tune.run(train, config=config, 
         resources_per_trial={'cpu': 2, 'gpu': 0.5}, 
         local_dir = '/content/drive/MyDrive/checkpoints', 
         checkpoint_freq=1)
'''

"\nconfig = {\n        \n      'lr': tune.grid_search([1e-4, 1e-3, 1e-2, 1e-1]),\n      'batch_size': tune.grid_search([512, 1024]),\n      'weight_decay': tune.grid_search([1e-3, 1e-2, 1e-1]),\n      'nb_units': [132, 64, 32, 16],\n      'momentum': tune.grid_search([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])\n\n}\n\n\nconfig = {\n        \n      'lr': tune.grid_search([1e-4]),\n      'batch_size': tune.grid_search([1024]),\n      'weight_decay': tune.grid_search([1e-1]),\n      'nb_units': tune.grid_search([[132, 64, 32, 16]]),\n      'momentum': tune.grid_search([0.2])\n\n}\n\ntune.run(train, config=config, \n         resources_per_trial={'cpu': 2, 'gpu': 0.5}, \n         local_dir = '/content/drive/MyDrive/checkpoints', \n         checkpoint_freq=1)\n"